In [6]:
from torchvision.models import squeezenet1_1
from models.utils import training
import torch
from torch import nn
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import torchvision.models as models
import sys,os
sys.path.append(os.getcwd())



workers = 0 if os.name == 'nt' else 4

In [7]:
data_dir = './data/test_data'
batch_size = 64
epochs = 1000
workers = 0 if os.name == 'nt' else 8

In [8]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))
print(torch.cuda.is_available())
print(torch.__version__)

Running on device: cuda:0
True
2.1.0+cu121


In [9]:
# from models.mtcnn import MTCNN
from facenet_pytorch import MTCNN
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [10]:
# dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
# dataset.samples = [
#     (p, p.replace(data_dir, data_dir + '_cropped'))
#         for p, _ in dataset.samples
# ]

# loader = DataLoader(
#     dataset,
#     num_workers=workers,
#     batch_size=batch_size,
#     collate_fn=training.collate_pil
# )

# for i,(x,y) in enumerate(loader):
#         mtcnn(x, save_path=y)
#         print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


dataset = datasets.ImageFolder(data_dir+'_cropped', transform=preprocess)

In [11]:
model_path = 'models/squeezenet1_1-b8a52dc0.pth'
squeeze = squeezenet1_1(pretrained = False)
squeeze.load_state_dict(torch.load(model_path))

for param in squeeze.parameters():
    param.requires_grad = False


final_conv = nn.Conv2d(512, dataset.class_to_idx.__len__(), kernel_size=1)
# nn.init.xavier_uniform_(final_conv.weight) # Xavier uniform initialization
nn.init.normal_(final_conv.weight, mean=0.0, std=0.01) # Normal initialization
nn.init.constant_(final_conv.bias, 0) # Initialize bias with a constant
squeeze.classifier[1] = final_conv
squeeze = squeeze.to('cuda')


for param in squeeze.classifier[1].parameters():
    param.requires_grad = True


# 解冻模型的最后两个Fire模块
for param in squeeze.features[-2:].parameters():
    param.requires_grad = True


print(final_conv.weight.shape)



# print(squeeze.state_dict())

# print(squeeze.classifier[1].state_dict())
# print(squeeze.features[0])




/home/lyy/anaconda3/envs/facenet/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/lyy/anaconda3/envs/facenet/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


torch.Size([5749, 512, 1, 1])


In [12]:
optimizer = optim.Adam(squeeze.parameters(), lr=0.008)
scheduler = MultiStepLR(optimizer, [5, 10])


img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [14]:

writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10
print('\n\n初始化')
print('-' * 10)
squeeze.eval()
training.pass_epoch(
    squeeze, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
   
)

for epoch in range(epochs):
    print('\n循环 {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)
    squeeze.train()
    training.pass_epoch(
        squeeze, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
 
    )

    squeeze.eval()
    training.pass_epoch(
        squeeze, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
     
    )
    

writer.close()



初始化
----------


Valid |    42/42   | loss:    8.7416 | fps: 1290.8907 | acc:    0.0007   

循环 1/1000
----------
Train |   166/166  | loss:    8.8780 | fps:  855.7125 | acc:    0.0309   
Valid |    42/42   | loss:    8.2424 | fps: 1307.3850 | acc:    0.0483   

循环 2/1000
----------
Train |   166/166  | loss:    8.2066 | fps:  861.6021 | acc:    0.0441   
Valid |    42/42   | loss:    8.1435 | fps: 1046.3645 | acc:    0.0594   

循环 3/1000
----------
Train |   166/166  | loss:    8.1407 | fps: 1096.7273 | acc:    0.0565   
Valid |    42/42   | loss:    8.1052 | fps:  949.3602 | acc:    0.0606   

循环 4/1000
----------
Train |   166/166  | loss:    8.1251 | fps:  965.5215 | acc:    0.0608   
Valid |    42/42   | loss:    8.1010 | fps: 1044.9603 | acc:    0.0649   

循环 5/1000
----------
Train |   166/166  | loss:    8.0807 | fps:  894.0585 | acc:    0.0639   
Valid |    42/42   | loss:    8.1066 | fps: 1268.5350 | acc:    0.0649   

循环 6/1000
----------
Train |   166/166  | loss:    8.0274 | fps:  927.0038 

KeyboardInterrupt: 